In [47]:
import sys
del sys.modules["mlml_hugginface"]

In [48]:
import pip
!pip install -e /home/berste/projects/language-learning-modelling/mlm-pipeline
!clear

Obtaining file:///home/berste/projects/language-learning-modelling/mlm-pipeline
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for mlml-hugginface (pyproject.toml) ... done
  Created wheel for mlml-hugginface: filename=mlml_hugginface-0.1.0-py3-none-any.whl size=1243 sha256=3cd9f4272d770c1fcf5da66fa6d13156ea782552e84d66e136a7e99b93ee0824
  Stored in directory: /tmp/pip-ephem-wheel-cache-6mtxiyu2/wheels/9a/4f/f2/38097cdc9e84254ee314e5dd685db7ba837b6041461c1743bb
Successfully built mlml-hugginface
  Attempting uninstall: mlml-hugginface
    Found existing installation: mlml-hugginface 0.1.0
    Uninstalling mlml-hugginface-0.1.0:
      Successfully uninstalled mlml-hugginface-0.1.0

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [52]:
from mlml_hugginface import Trainer

In [53]:
import os
os.getcwd()

'/home/berste/projects/language-learning-modelling/mlm-pipeline/notebooks'

In [54]:
trainer = Trainer("../run_configs/train.json")


'>>> Review: She does gardening in the morning and sets the table at 6 pm.'

'>>> Review: The ticket price start at 50 euros and it allowed you to see all the festival and also pays for the camping.'

'>>> Review: It tells the story of a young man who goes to an island with her parents for the holidays.'


OSError: models/distilbert-base-uncased is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [1]:
trainer.train()

NameError: name 'trainer' is not defined

In [7]:
trainer.collated_dataset["train"][0]

{'input_ids': [101,
  103,
  12524,
  1045,
  2572,
  8025,
  26092,
  3756,
  2013,
  2026,
  103,
  3573,
  2138,
  1997,
  2035,
  1996,
  6704,
  2008,
  5129,
  2009,
  2043,
  2009,
  2001,
  103,
  2207,
  1999,
  3476,
  1012,
  1045,
  2036,
  2657,
  2008,
  2012,
  2034,
  2009,
  2001,
  8243,
  2011,
  103,
  1012,
  1055,
  1012,
  8205,
  103,
  2009,
  2412,
  2699,
  2000,
  4607,
  2023,
  2406,
  1010,
  3568,
  2108,
  1037,
  5470,
  1997,
  103,
  2641,
  1000,
  6801,
  1000,
  1045,
  2428,
  2018,
  2000,
  2156,
  2023,
  2005,
  103,
  1012,
  1026,
  7987,
  103,
  103,
  1026,
  7987,
  1013,
  1028,
  103,
  5436,
  2003,
  8857,
  2105,
  1037,
  2402,
  4467,
  3689,
  3076,
  2315,
  14229,
  103,
  4122,
  2000,
  4553,
  2673,
  103,
  2064,
  2055,
  2166,
  1012,
  1999,
  3327,
  2016,
  7276,
  2000,
  3579,
  2014,
  3086,
  2015,
  2000,
  2437,
  2070,
  103,
  1997,
  4516,
  2006,
  2054,
  1996,
  2779,
  25430,
  14728,
  103,
  2055,
  305

In [13]:
trainer.collated_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 50000
    })
})

In [18]:
import transformers as T

batch_size = 4
# Show the training loss with every epoch
logging_steps = len(trainer.collated_dataset["train"]) // batch_size
model_name = trainer.config['MODEL_CHECKPOINT'].split("/")[-1]

training_args = T.TrainingArguments(
    output_dir=f"{model_name}-finetuned-imdb",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    fp16=False,
    logging_steps=logging_steps,
)

hf_trainer = T.Trainer(
    model=trainer.model,
    args=training_args,
    train_dataset=trainer.collated_dataset["train"],
    eval_dataset=trainer.collated_dataset["test"],
    tokenizer=trainer.tokenizer,
)


In [17]:
hf_trainer.train()

TypeError: DistilBertForMaskedLM(
  (activation): GELUActivation()
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): D

In [13]:
from transformers import (
    AutoModelForMaskedLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
)

mlm_collator = DataCollatorForLanguageModeling(
            tokenizer=trainer.tokenizer,
            mlm=True,
            mlm_probability=trainer.config['MLM_PROBABILITY'],
            return_tensors="pt",
        )


In [142]:
#collator(trainer.tokenized_dataset['train'])
# mlm_collator([d for d in trainer.tokenized_dataset['train']])
mlm_collator([d['input_ids'] for d in trainer.tokenized_dataset['train']])

{'input_ids': tensor([[  101,  1045, 12524,  ...,     0,     0,     0],
         [  101,  1000,  1045,  ...,     0,     0,     0],
         [  101,  2065,  2069,  ...,     0,     0,     0],
         ...,
         [  101,   103,   103,  ...,     0,     0,     0],
         [  101,   103,  1996,  ...,  2108,   103,   102],
         [  101,  1996,  2466,  ...,     0,     0,     0]]),
 'labels': tensor([[-100, -100, -100,  ..., -100, -100, -100],
         [-100, -100, -100,  ..., -100, -100, -100],
         [-100, -100, -100,  ..., -100, -100, -100],
         ...,
         [-100, 2023, 2143,  ..., -100, -100, -100],
         [-100, 1005, -100,  ..., -100, 8487, -100],
         [-100, -100, -100,  ..., -100, -100, -100]])}

In [14]:
from transformers import DefaultDataCollator

In [19]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# This is new
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/bernardostearns/Library/Caches/pypoetry/virtualenvs/mlml-hugginface-YjjkMAzc-py3.11/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
batch

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  2023,  2607,  2003,  6429,   999,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'labels': tensor([1, 1])}

In [21]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [22]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})